## Package

In [4]:
!pip install langchain-chroma
!pip install langchain
!pip install langchain_community
!pip install langchainhub
!pip install -qU langchain-openai

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ------ --------------------------------- 10.2/67.3 kB ? eta -:--:--
     ----------------- -------------------- 30.7/67.3 kB 640.0 kB/s eta 0:00:01
     ---------------------------------- --- 61.4/67.3 kB 465.5 kB/s eta 0:00:01
     -------------------------------------- 67.3/67.3 kB 456.2 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/526.8 kB ? eta -:--:--
   ------ --------------------------------- 92.2/526.8 kB 1.7 MB/s eta 0:00:01
   ------ --------------------------------- 92.2/526.8 kB 1.7 MB/s eta 0:00:01
   -------- ------------------------------- 112.6/526.8 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.2 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.2 which is incompatible.


In [1]:
import os

os.environ["OPENAI_API_KEY"] = "Your_OPENAI_API_KEY"

# 1. Get a Data Loader

In [2]:
from langchain.document_loaders import DirectoryLoader

In [4]:
pwd=os.getcwd()
loader=DirectoryLoader(path=pwd,glob="t_data/*.csv",use_multithreading=True)
docs=loader.load()
print(len(docs))

2


: 

# 2. Convert data to Vector Database


In [13]:

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


In [22]:
Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings(model="text-embedding-3-large"), persist_directory="./chroma_db")

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [27]:
print (f'Now you have {len(split_docs)} documents')

Now you have 268 documents


In [28]:
Chroma.from_documents(documents=split_docs, embedding=OpenAIEmbeddings(model="text-embedding-3-large"), persist_directory="./chroma_db")

# 3. Make a RAG pipeline


In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [30]:
llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"))

In [31]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

## test

In [32]:
question = "請推薦我一個台南的最棒民宿，並告訴我價格還有聯絡資料及地址"
result = qa_chain({"query": question })

In [33]:
result

{'query': '請推薦我一個台南的最棒民宿，並告訴我價格還有聯絡資料及地址',
 'result': '推薦您「長長99」，這是位於台南東區的獨棟老屋民宿，地址是臺南市東區長榮路二段32巷99號，價格範圍為2000至2800新台幣。聯絡電話是886-933-666272。'}

# pandas ai

In [ ]:
# 確認 'Region' 欄位中沒有缺失值
allspot_data_clean = allspot_data.dropna(subset=['Region'])

# 計算每個 Region 的資料筆數
region_counts = allspot_data_clean['Region'].value_counts()

# 顯示結果
print(region_counts)